In [334]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [335]:
data = pd.read_table('botswana.tsv')
print data.shape
data.head()

(4361, 15)


,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [336]:
data.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,2079.000000,1956.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,20.686388,5.144683,0.516625,0.140202,0.701766,0.092911,0.275815
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,5.002383,4.803028,0.499781,0.347236,0.457535,0.290341,0.446975
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,23.000000,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


$\textbf{№1}$

In [337]:
data['religion'].value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

$\textbf{№2}$

In [338]:
data.dropna().shape[0]

1834

$\textbf{№3}$

In [339]:
data['nevermarr'] = map(lambda x: 1 if x else 0, pd.isna(data.agefm))
ind = [i for i in xrange(data.shape[0]) if data['nevermarr'][i] == 1]
data['heduc'].iloc[ind] = -1
data['agefm'].fillna(0, inplace=True)
data.drop('evermarr', axis=1, inplace=True)
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [340]:
data['heduc'].isna().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

$\textbf{№4}$

In [341]:
data['idlnchld_noans'] = map(lambda x: 1 if x else 0, pd.isna(data.idlnchld))
data['heduc_noans'] = map(lambda x: 1 if x else 0, pd.isna(data.heduc))
data['usemeth_noans'] = map(lambda x: 1 if x else 0, pd.isna(data.usemeth))

In [343]:
data['idlnchld'].fillna(-1, inplace=True)
data['usemeth'].fillna(-1, inplace=True)
data['heduc'].fillna(-2, inplace=True)
data.dropna(inplace=True, subset=['knowmeth', 'electric', 'radio', 'tv', 'bicycle'])
data.shape[0] * data.shape[1]

78264

$\textbf{№5}$

In [389]:
model = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth +'\
             'usemeth + agefm + heduc + urban + electric + radio +'\
             'tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
              data=data)
fitted = model.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Wed, 03 Jan 2018   Prob (F-statistic):               0.00
Time:                        21:57:28   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

$\textbf{№6}$

3

$\textbf{№7}$

In [370]:
print 'p={}'.format(sms.het_breushpagan(fitted.resid, fitted.model.exog)[1])

p=1.14529276334e-225


/Users/andrey/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [383]:
model = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth +'\
             'usemeth + agefm + heduc + urban + electric + radio +'\
             'tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
              data=data)
fitted = model.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Wed, 03 Jan 2018   Prob (F-statistic):               0.00
Time:                        21:49:30   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

$\textbf{№8}$

In [384]:
model_del_ftrs = smf.ols('ceb ~ age + educ + idlnchld + knowmeth +'\
             'usemeth + agefm + heduc + urban + electric +'\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
              data=data)
fitted = model_del_ftrs.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Wed, 03 Jan 2018   Prob (F-statistic):               0.00
Time:                        21:49:30   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [385]:
print "F=%f, p=%f, k1=%f" % model.fit().compare_f_test(model_del_ftrs.fit())

F=0.919236, p=0.467231, k1=5.000000


$\textbf{№9}$

In [386]:
model_del_try = smf.ols('ceb ~ age + educ + idlnchld + knowmeth +'\
             'agefm + heduc + urban + electric +'\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans', 
              data=data)
fitted = model_del_try.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Wed, 03 Jan 2018   Prob (F-statistic):               0.00
Time:                        21:49:32   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [387]:
print model_del_ftrs.fit().compare_f_test(model_del_try.fit())

(92.890582301098021, 3.1552009480371243e-40, 2.0)


40

$\textbf{№10}$

In [388]:
model_del_ftrs = smf.ols('ceb ~ age + educ + idlnchld + knowmeth +'\
             'usemeth + agefm + heduc + urban + electric +'\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
              data=data)
fitted = model_del_ftrs.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Wed, 03 Jan 2018   Prob (F-statistic):               0.00
Time:                        21:49:33   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

NOT:

nevermarr, 0.63v, age+, idlchl + noans

without knowmeth

withou hnowmeth, var
